In [6]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/catanaranjo/fraudDetection/master/dev.csv") 
dfo = pd.read_csv("https://raw.githubusercontent.com/catanaranjo/fraudDetection/master/oot0.csv")

In [7]:
df['dataset'] = 'train'
dfo['dataset'] = 'test'
dfo['ob_target'] = 0
trainTestDF= df.append(dfo)

In [8]:
numericList = []
for i in range(65,82):
    numericList.append('if_var_' + str(i))

In [9]:
import numpy as np
import warnings; warnings.simplefilter('ignore')
def change_outliers(dataframe):
    list = []
    list = numericList
    for column in list:
        q1 = dataframe[column].quantile(0.25)
        q3 = dataframe[column].quantile(0.75)
        iqr = q3-q1 #Interquartile range
        highest = q3 + 1.5*iqr
        lowest = q1 - 1.5*iqr
        median = dataframe.loc[dataframe[column]<q3 + 1.5*iqr, column].median()
        outliersUp = (dataframe[column]).abs() > round(q3 + 1.5*iqr)
        outliersDown = (dataframe[column]).abs() < round(q1 - 1.5*iqr)
        dataframe[column][outliersUp] = np.nan
        dataframe[column].fillna(highest, inplace=True)
        dataframe[column][outliersDown] = np.nan
        dataframe[column].fillna(lowest, inplace=True)

numericDF = trainTestDF[numericList]
change_outliers(numericDF)

In [10]:
categorical_transformed_trainDF = trainTestDF[trainTestDF['dataset']=='train']
categorical_transformed_testDF = trainTestDF[trainTestDF['dataset']=='test']

# y_train = trainTestDF['ob_target']
y_train = categorical_transformed_trainDF['ob_target']

drops = ['ob_target','dataset']
categorical_ohe_train = categorical_transformed_trainDF.loc[:,~categorical_transformed_trainDF.columns.isin(drops)]
categorical_ohe_test = categorical_transformed_testDF.loc[:,~categorical_transformed_testDF.columns.isin(drops)]

X_train = pd.get_dummies(categorical_ohe_train) 
X_test = pd.get_dummies(categorical_ohe_test)
X_train.head()

,id,ib_var_1,ib_var_2,ib_var_3,ib_var_4,ib_var_5,ib_var_6,ib_var_7,ib_var_8,ib_var_9,...,if_var_72,if_var_73,if_var_74,if_var_75,if_var_76,if_var_77,if_var_78,if_var_79,if_var_80,if_var_81
0,1,0,1,0,0,1,1,0,0,0,...,6,0.800,0.0,6.0,5.0,0.500000,9.4634,5140.0,0.766667,1
1,2,0,1,0,0,0,1,0,0,0,...,2,0.925,5.0,8.0,5.0,0.400000,7.6341,2570.0,0.700000,4
2,3,0,0,0,0,1,1,0,0,0,...,3,0.800,3.0,10.0,6.0,0.700000,11.1707,5140.0,0.666667,2
3,4,0,1,0,1,1,1,0,0,0,...,6,0.825,5.0,5.0,6.0,0.433333,8.0488,1028.0,0.766667,3
4,5,0,0,0,0,0,1,0,0,0,...,3,0.800,0.0,11.0,5.0,0.700000,5.5854,5140.0,0.733333,3


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

baseline_rf = RandomForestClassifier(n_jobs=-1,random_state=42,n_estimators=4000,criterion='entropy',max_features='log2',
                                    bootstrap=True, min_samples_split=3,min_samples_leaf=1,oob_score=True)

baseline_rf.fit(X_train,y_train)
y_pred_rf = baseline_rf.predict_proba(X_train)[:,1]

 
gini_score = 2*roc_auc_score(y_train, y_pred_rf)-1
print ("GINI DEVELOPMENT=", gini_score)

GINI DEVELOPMENT= 1.0
